<a href="https://colab.research.google.com/github/moaaz12-web/RAG-using-Langchain-OpenAI-and-Huggingface/blob/main/RAG_with_Ensemble_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build RAG pipeline using Open Source Large Languages

This notebook uses traditional Document Question Answering with different retireval techniques

## Installation

In [1]:
!pip install --upgrade --quiet  rank_bm25 > /dev/null

In [ ]:
! pip install openai tiktoken
!pip install langchain faiss-cpu sentence-transformers chromadb
! pip install --upgrade --quiet  "unstructured[pdf]"


## Reading the data

In [3]:
from langchain_community.document_loaders import UnstructuredFileLoader
loader = UnstructuredFileLoader("/content/Copie authentique Bail définitif Carcassonne.pdf")
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Text Splitting into chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=80)
chunks = text_splitter.split_documents(docs)

In [5]:
chunks[4]

Document(page_content='La Société dénommée BV BUROSTOC est ici représentée par Mme Karine RENERRE, clerc de Notaire domiciliée à Carcassonne 29 B Bld Marcou, agissant en vertu des pouvoirs qui lui ont été conférés aux termes d’un pouvoir sous seing privée à LES CLAYES SOUS BOIS le 3 juin 2013 par Mr Jean Baptiste EUDELINE directeur réseaux, ayant lui-même reçu tous pouvoirs par le Président Directeur Général de ladite société Monsieur Bruno PEYROLES, en vertu d’un pouvoirs sous seing-privée en date à CLAYES SOUS BOIS du 16 mai', metadata={'source': '/content/Copie authentique Bail définitif Carcassonne.pdf'})

In [6]:
len(chunks)

115

## Embeddings

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="YOUR_KEY")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Vector Store - FAISS

In [8]:
from langchain.vectorstores.faiss import FAISS
vectorstore = FAISS.from_documents(chunks, embeddings)

In [9]:
query =  "What is the address of the premises in the commercial lease?"
search = vectorstore.similarity_search(query)
search

[Document(page_content='Article 1 – Etat des lieux\n\nLE PRENEUR prendra les lieux loués conformément au descriptif technique et plans annexés aux présentes, l’immeuble est livré brut (clos et couvert seulement réalisé par le Bailleur) en même temps que les parkings et les espaces verts et tous les autres éléments communs selon descriptif visé ci-dessus.\n\nUn état des lieux a été réalisé par Me HADJADJ Huissier de Justice à Carcassonne le 12 avril 2013, dont une copie est jointe et annexée aux présentes après mention.', metadata={'source': '/content/Copie authentique Bail définitif Carcassonne.pdf'}),
 Document(page_content="1°-1°- CONCERNANT\n\nPRENEUR CONCERNANT LELE PRENEUR\n\nLe présent bail est fait sous les charges et conditions ordinaires et de droit en pareille matière et notamment sous celles suivantes que LE PRENEUR s'oblige à bien et fidèlement exécuter à peine de tous dépens et dommages-intérêts et même de résiliation des présentes, si bon semble au BAILLEUR, savoir :\n\nA

## LLM - OpenAI

In [10]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key="YOUR_API_KEY")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Retrievers

### TYPICAL RETRIEVER


In [11]:
retriever = vectorstore.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 4}
)

### MULTI-QUERY RETREIVER

In [13]:
from langchain.retrievers.multi_query import MultiQueryRetriever
retriever_mquery = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

### BM25Retriever

In [12]:
from langchain.retrievers import BM25Retriever
bm25_retriever=BM25Retriever.from_documents(docs)
bm25_retriever.k=1


### Ensemble retriever

In [14]:
from langchain.retrievers import EnsembleRetriever

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever_mquery, retriever], weights=[0.8, 0.2]
)

## PROMPT TEMPLATE

In [15]:
# prompt_str = """You are provided some context below
# and then there is a question.
# You need to answer the question using that context.
# If you don't know the answer or can't find it, say SORRY I DONT KNOW:

# Context: {context}

# Question: {question}

# Answer: """

prompt_str ="""Vous êtes fourni avec un contexte ci-dessous
et ensuite il y a une question.
Vous devez répondre à la question en utilisant ce contexte.
Si vous ne connaissez pas la réponse ou ne pouvez pas la trouver, dites DÉSOLÉ, JE NE SAIS PAS :

Contexte : {contexte}

Question : {question}

Réponse : """

from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(prompt_str)

## OUTPUT PARSER

In [16]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

## BUILDING CHAIN USING LECL

In [17]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

In [18]:
retrieval = RunnableParallel(
    {"contexte": ensemble_retriever, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | llm | output_parser

Sample questions in french


what is the rent for the commercial lease?

What is the name of Lessor / Name of Landlord / Address of the premises?

In [21]:
out = chain.invoke("Quel est le loyer inscrit dans le contrat?")
print(out)

Le loyer inscrit dans le contrat est de SOIXANTE CINQ MILLE EUROS (65.000,00 €) Hors Taxes par an.


Q1: Quel est le nom du bailleur?

A1: Le nom du bailleur est SC LES AGRIERS.


Q2: Quel est le nom de la société locataire?

A2:BV BUROSTOC

Q3: Quel est le loyer inscrit dans le contrat?

A3: Le loyer inscrit dans le contrat est de SOIXANTE CINQ MILLE EUROS (65.000,00 €) Hors Taxes par an.

